In [ ]:
import pandas as pd

: 

In [ ]:
data = pd.read_csv('../dataset/loan_approval_dataset.csv')
print(data.columns)

Index(['loan_id', ' no_of_dependents', ' education', ' self_employed',
       ' income_annum', ' loan_amount', ' loan_term', ' cibil_score',
       ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', ' loan_status'],
      dtype='object')


In [ ]:
data.columns = data.columns.str.strip()

In [ ]:
print(data.isnull().sum())
data.dropna(inplace=True)

loan_id                     0
no_of_dependents            0
education                   0
self_employed               0
income_annum                0
loan_amount                 0
loan_term                   0
cibil_score                 0
residential_assets_value    0
commercial_assets_value     0
luxury_assets_value         0
bank_asset_value            0
loan_status                 0
dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_cols = ["education", "self_employed", "loan_status"]
le = LabelEncoder()

for col in label_cols:
    data[col] = le.fit_transform(data[col])

In [ ]:
x = data.drop(["loan_id","loan_status"], axis=1)
y = data["loan_status"]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred))

Accuracy: 98.13%
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       536
           1       0.98      0.97      0.97       318

    accuracy                           0.98       854
   macro avg       0.98      0.98      0.98       854
weighted avg       0.98      0.98      0.98       854



In [ ]:
import joblib
import os

os.makedirs("../model", exist_ok=True)
joblib.dump(model, "../model/model.pkl")
joblib.dump(scaler, "../model/preprocessor.pkl")

['../model/preprocessor.pkl']